<p><strong><font size="6">Accessibility to health centers</font></strong></p>

This python code implement the method developed by UNamur and ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys

** Add folder with SCR provided belong to this notebook**

In [3]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [4]:
run ../SRC/config.py

In [5]:
print config_parameters

{'permanent_mapset': 'PERMANENT', 'locationepsg': '32628', 'outputfolder': '../Results/SEN', 'gisdb': '../GRASSDATA', 'location': 'shedecides', 'time_limits': ('30', '60', '120', '240', '360', '480'), 'PYTHONLIB': '/usr/lib/python2.7', 'njobs': 3, 'resolution': '100', 'GISBASE': '/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu'}


In [6]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [7]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/script:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 2084 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/2012,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/2012 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD_LI

** GRASS GIS Python libraries **

In [8]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

** Other functions**

In [9]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset

In [10]:
# Import functions for processing time information
from processing_time import start_processing, print_processing_time

In [11]:
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name

In [12]:
# Import function that clip multiple raster according to extention of a vector layer
from clip_multiple_raster import clip_multiple_raster

In [13]:
# Import function that check and create folder
from mkdir import check_create_dir

In [14]:
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**Create list for temporary layers management**

In [15]:
# Create list for storing name of intermediates layers
TMP_rast = []
TMP_vect = []

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Preprocessing

**Creation of GRASSDATA in WGS84 (Lat/Long)**

In [16]:
# Check if the GRASS GIS database exists and create it if not
check_gisdb(config_parameters["gisdb"])
# Check if the location exists and create it if not, with the CRS defined by the epsg code 
check_location(config_parameters["gisdb"],"Worldpop_transform","4326")
# Check if the mapset exists and create it if not
check_mapset(config_parameters["gisdb"],"Worldpop_transform",config_parameters["permanent_mapset"])
# Change the current working GRASS GIS session mapset
working_mapset(config_parameters["gisdb"],"Worldpop_transform",config_parameters["permanent_mapset"])

GRASSDATA folder created in '../GRASSDATA'
Location 'Worldpop_transform' created
'PERMANENT' mapset already exists in location 'Worldpop_transform'
You are now working in mapset 'PERMANENT'


**WOCBA**

In [17]:
# Import raster
gscript.run_command('r.in.gdal', overwrite=True, input=data['WOCBA_PPP'][1], output=data['WOCBA_PPP'][0])
# Define region base on this layer
gscript.run_command('g.region', flags='s', raster=data['WOCBA_PPP'][0])

0

In [18]:
# Compute density - Population per hectare (pph)
formula = "%s=%s/(area()/10000)"%(data['WOCBA_PPH'],data['WOCBA_PPP'][0])
gscript.mapcalc(formula, overwrite=True)

**POPULATION**

In [19]:
# Import raster
gscript.run_command('r.in.gdal', overwrite=True, input=data['POP_PPP'][1], output=data['POP_PPP'][0])
# Define region base on this layer
gscript.run_command('g.region', raster=data['POP_PPP'][0])

0

In [20]:
# Compute density - Population per hectare (pph)
formula = "%s=%s/(area()/10000)"%(data['POP_PPH'],data['POP_PPP'][0])
gscript.mapcalc(formula, overwrite=True)

**HEALTH CENTERS**

Please ensure that the .csv file has pipe as column delimiter (|), point as decimal limiter for X/Y columns.

The columns should be as follow, otherwise the code should be adapted: 'x double precision, y double precision, short_name varchar, level integer'.

TODO: Add check if points fall outside of current computational region.

In [21]:
# Import points
gscript.run_command('g.region', flags='d')
gscript.run_command('v.in.ascii', flags='r', overwrite=True, input=data['HC'][1], 
                    output=data['HC'][0], skip=1, 
                    columns='x double precision, y double precision, short_name varchar, level integer')

0

**LULC - GLobcover**

In [22]:
# Import raster
gscript.run_command('r.in.gdal', overwrite=True, input=data['LULC'][1], output=data['LULC'][0])

0

**ROADS**

In [23]:
# Import vector
gscript.run_command('v.in.ogr', overwrite=True, input=data['ROADS'][1], output=data['ROADS'][0])

0

# Processing : Computation of accessibility to health centers

## Launch GRASS GIS session

In GRASS GIS, original data are usually imported and stored in the "PERMANENT" mapset (automatically created when creating a new location).

In [24]:
# Check if the GRASS GIS database exists and create it if not
check_gisdb(config_parameters["gisdb"])
# Check if the location exists and create it if not, with the CRS defined by the epsg code 
check_location(config_parameters["gisdb"],config_parameters["location"],config_parameters["locationepsg"])
# Check if the mapset exists and create it if not
check_mapset(config_parameters["gisdb"],config_parameters["location"],config_parameters["permanent_mapset"])
# Change the current working GRASS GIS session mapset
working_mapset(config_parameters["gisdb"],config_parameters["location"],config_parameters["permanent_mapset"])

GRASSDATA folder already exist
Location 'shedecides' created
'PERMANENT' mapset already exists in location 'shedecides'
You are now working in mapset 'PERMANENT'


# Import data / Preparation of data

In [25]:
# Saving current time for processing time management
begintime_importdata=start_processing()

## Import administrative units

In [26]:
# Import administrative units
gscript.run_command('v.import', overwrite=True, input=data['admin'][1], output=data['admin'][0])

0

## Create study area from administrative units

In [27]:
# Create a name for temporary layer
tmp_layer = random_layer_name()
# Create new vector layer based on SQL query on the attributes
gscript.run_command('v.extract', overwrite=True, input=data['admin'][0], where="ID_1 in (1,13,8,2,3)", output=tmp_layer)
#gscript.run_command('v.extract', overwrite=True, input=data['admin'][0], where="ID_0=201", output=tmp_layer)
# Dissolve result from previous according to common field
gscript.run_command('v.dissolve', overwrite=True, input=tmp_layer, column="ID_0", output="Study_area")
# Remove temporary layer
gscript.run_command('g.remove', flags='f', type='vector', name=tmp_layer)

0

## Import raster (LULC, Population, WOCBA) for study area

In [28]:
# Define computational region based on the Study_area vector layer
gscript.run_command('g.region', vector="Study_area")

0

**WOCBA**

In [29]:
# Reproject layer from location in WGS84
gscript.run_command('r.proj', overwrite=True, location="Worldpop_transform", method='bicubic',
                    input=data['WOCBA_PPH'], output=data['WOCBA_PPH'],
                    resolution=config_parameters['resolution'])

# Save default computational region (CR) based on the WOCBA
gscript.run_command('g.region', flags='s', raster=data['WOCBA_PPH'])

# Compute Population per pixel (ppp)
formula = "%s=%s"%(data['WOCBA_PPP'][0],data['WOCBA_PPH'])
gscript.mapcalc(formula, overwrite=True)

# Remove pph raster
gscript.run_command('g.remove', flags='f', type='raster', name=data['WOCBA_PPH'])

0

**POPULATION**

In [30]:
# Reproject layer from location in WGS84
gscript.run_command('r.proj', overwrite=True, location="Worldpop_transform", method='bicubic',
                    input=data['POP_PPH'], output=data['POP_PPH'],
                    resolution=config_parameters['resolution'])

# Save default computational region (CR) based on the WOCBA
gscript.run_command('g.region', flags='s', raster=data['POP_PPH'])

# Compute Population per pixel (ppp)
formula = "%s=%s"%(data['POP_PPP'][0],data['POP_PPH'])
gscript.mapcalc(formula, overwrite=True)

# Remove pph raster
gscript.run_command('g.remove', flags='f', type='raster', name=data['POP_PPH'])

0

**LULC - GLobcover**

In [31]:
# Reproject layer from location in WGS84
gscript.run_command('r.proj', overwrite=True, location="Worldpop_transform", method='nearest',
                    input=data['LULC'][0], output=data['LULC'][0],
                    resolution=config_parameters['resolution'])
TMP_rast.append(data['LULC'][0])

## Import vector (roads) for study area

In [32]:
# Instal required add-on if not yet installed
check_install_addon("v.clip")

v.clip is already installed on your computer


In [33]:
# Create a name for temporary layer
tmp_layer = random_layer_name()
# Reproject layer from location in WGS84
gscript.run_command('v.proj', overwrite=True, location="Worldpop_transform",
                    input=data['ROADS'][0], output=tmp_layer)

0

In [34]:
# Clip vector layer
gscript.run_command('v.clip', overwrite=True, input=tmp_layer, clip="Study_area", output=data['ROADS'][0])
TMP_vect.append(data['ROADS'][0])
# Remove temporary layer
gscript.run_command('g.remove', flags='f', type='vector', name=tmp_layer)

0

## Clip raster to match study area extent

Please notice that WOCBA is used as reference for following steps (spatial extent and pixel alignement).

In [35]:
# Instal required add-on if not yet installed
check_install_addon("r.clip")

r.clip is already installed on your computer


In [36]:
# Define the list with names of raster layers to clip
raster_to_clip = [data['WOCBA_PPP'][0],data['POP_PPP'][0],data['LULC'][0]]
print "\n".join(raster_to_clip)

WOCBA_PPP
POP_PPP
LULC


In [37]:
# Define computational region based default region
gscript.run_command('g.region', flags='d')
# Define MASK and get a copy
gscript.run_command('r.mask', overwrite=True, vector='Study_area')
gscript.run_command('g.copy', overwrite=True, raster='MASK,Study_area')
# Clip multiple rasters
clip_multiple_raster(raster_to_clip, overwrite=True, resample=True, n_jobs=config_parameters['njobs'])
# Rename raster to overwrite the original one
[gscript.run_command('g.rename', overwrite=True, raster='%s_clip,%s'%(name,name)) for name in raster_to_clip]
# Remove MASK
gscript.run_command('r.mask', flags='r')

'WOCBA_PPP' has been cliped.
'POP_PPP' has been cliped.
'LULC' has been cliped.


0

## Health facilities (HC)

In [38]:
# Create a name for temporary layer
tmp_layer = random_layer_name()
# Reproject layer from location in WGS84
gscript.run_command('v.proj', overwrite=True, location="Worldpop_transform", input=data['HC'][0], output=tmp_layer)

0

In [39]:
# Select point into study area (create new layer)
gscript.run_command('v.select', overwrite=True, ainput=tmp_layer, binput="Study_area", 
                    output=data['HC'][0], operator="within")
TMP_vect.append(data['HC'][0])
# Remove temporary layer
gscript.run_command('g.remove', flags='f', type='vector', name=tmp_layer)

# Create two new sub-layer based on level of HC (HCL1 ; HCL2)
gscript.run_command('v.extract', overwrite=True, input=data['HC'][0], 
                    where="level=1", output='%sL1'%data['HC'][0])
gscript.run_command('v.extract', overwrite=True, input=data['HC'][0], 
                    where="level=2", output='%sL2'%data['HC'][0])

# Rename HC layer that contain both levels
gscript.run_command('g.rename', overwrite=True, vector="%s,%sL1L2"%(data['HC'][0],data['HC'][0]))

0

## Convert roads into raster

In [40]:
# Define computational region based default region
gscript.run_command('g.region', flags='d')
# Rasterize buffered roads layer
gscript.run_command('v.to.rast', overwrite=True, input=data['ROADS'][0], output=data['ROADS'][0], use='val')

0

## Reclassify LULC

In [41]:
for season in ("WS","DS"):
    # Create a name for temporary layer
    tmp_layer = random_layer_name() 
    # Define computational region based default region
    gscript.run_command('g.region', flags='d')
    # Reclassify raster
    gscript.run_command('r.reclass', overwrite=True, input=data['LULC'][0], 
                        output=tmp_layer, rules=rule_file['globcover_%s'%season])
    # Create hard copy of the reclassified raster
    formula = "%s_%s=%s"%(data['LULC'][0],season,tmp_layer)
    gscript.mapcalc(formula, overwrite=True)
    TMP_rast.append(tmp_layer)

In [42]:
## Print processing time
print_processing_time(begintime_importdata ,"Data imported in ")

'Data imported in 35.0 seconds'

# Methodology

In [43]:
# Saving current time for processing time management
begintime_processing=start_processing()

## Create raster of velocity

**LULC**

In [44]:
# Create velocity raster
for season in ("WS","DS"):
    # Create a name for temporary layer
    tmp_layer = random_layer_name() 
    # Define computational region based default region
    gscript.run_command('g.region', flags='d')
    # Reclassify raster
    gscript.run_command('r.reclass', overwrite=True, input="%s_%s"%(data['LULC'][0],season), 
                        output=tmp_layer, rules=rule_file['Velocity_LULC'])
    # Create hard copy of the reclassified raster
    formula = "velocity_%s_%s=%s"%(data['LULC'][0],season,tmp_layer)
    gscript.mapcalc(formula, overwrite=True)
    # Remove temporary layer
    TMP_rast.append(tmp_layer)

**ROADS**

In [45]:
# Create velocity raster 
for car in ("WC","NC"):
    # Define computational region based default region
    gscript.run_command('g.region', flags='d')
    # Create velocity raster
    formula = "velocity_%s_%s = if(%s==1,%s,null())"%(data['ROADS'][0],car,data['ROADS'][0],roads_veloc[car])
    gscript.mapcalc(formula, overwrite=True)

## Combine two rasters of velocity together

In [46]:
# Create a list for saving layer name
veloc_raster = []

# Create all possible combinations
for season in ("WS","DS"):
    for car in ("WC","NC"):
        # Define computational region based default region
        gscript.run_command('g.region', flags='d')
        # Create velocity raster
        veloc_LULC = "velocity_%s_%s"%(data['LULC'][0],season)
        veloc_ROADS = "velocity_%s_%s"%(data['ROADS'][0],car)
        veloc_combined = "velocity_%s_%s"%(car,season)
        formula = "%s=if(isnull(%s),%s,%s)/100.00"%(veloc_combined,veloc_ROADS,veloc_LULC,veloc_ROADS)
        gscript.mapcalc(formula, overwrite=True)
        veloc_raster.append(veloc_combined)
        TMP_rast.append(veloc_LULC)
        TMP_rast.append(veloc_ROADS)
        TMP_rast.append(veloc_combined)

## Calculate raster cost distance

The computation of cost distance raster is performed here using [r.cost](https://grass.osgeo.org/grass76/manuals/r.cost.html).

In [47]:
# Create a list for saving layer name
cost_raster = []

# Create all cost distance raster
for veloc_rast in veloc_raster:
    for hclevel in ("L1L2","L2"):
        # Define name of the output layer
        output_layer = "Pre_CD_HC%s_%s"%(hclevel,veloc_rast[-5:])
        # Define computational region based default region
        gscript.run_command('g.region', flags='d')
        # Compute cost distance raster -'k' for Knight's move
        gscript.run_command('r.cost', flags='k', overwrite=True,
                            input=veloc_rast, output=output_layer, 
                            start_points="%s%s"%(data['HC'][0],hclevel), memory="5000")
        print "Layer '%s' created."%output_layer
        cost_raster.append(output_layer)
        TMP_rast.append(output_layer)

Layer 'Pre_CD_HCL1L2_WC_WS' created.
Layer 'Pre_CD_HCL2_WC_WS' created.
Layer 'Pre_CD_HCL1L2_NC_WS' created.
Layer 'Pre_CD_HCL2_NC_WS' created.
Layer 'Pre_CD_HCL1L2_WC_DS' created.
Layer 'Pre_CD_HCL2_WC_DS' created.
Layer 'Pre_CD_HCL1L2_NC_DS' created.
Layer 'Pre_CD_HCL2_NC_DS' created.


In [48]:
# Create a list for saving layer name
isochrone_layers = []

# Create isochrones rasters
for cost_rast in cost_raster:
    # Define name of the output layer
    output_layer = "Isochrones_%s"%cost_rast[7:]
    # Define computational region based default region
    gscript.run_command('g.region', flags='d')
    # Create accessibility raster (time to travel)
    formula_prefix = "%s="%output_layer
    formula_suffix = "null()"
    for time in config_parameters['time_limits']:
        formula_prefix += "if(%s<=%s,%s,"%(cost_rast,time,time)
        formula_suffix += ")"
    gscript.mapcalc(formula_prefix+formula_suffix, overwrite=True)
    print "Layer '%s' created."%output_layer
    isochrone_layers.append(output_layer)

Layer 'Isochrones_HCL1L2_WC_WS' created.
Layer 'Isochrones_HCL2_WC_WS' created.
Layer 'Isochrones_HCL1L2_NC_WS' created.
Layer 'Isochrones_HCL2_NC_WS' created.
Layer 'Isochrones_HCL1L2_WC_DS' created.
Layer 'Isochrones_HCL2_WC_DS' created.
Layer 'Isochrones_HCL1L2_NC_DS' created.
Layer 'Isochrones_HCL2_NC_DS' created.


In [49]:
# Convert isochrone rasters to vector layers
for isochrone in isochrone_layers:
    gscript.run_command('g.region', flags='d')
    gscript.run_command('r.to.vect', flags='v', overwrite=True,
                        input=isochrone, output=isochrone, type="area")

## Overlay population & calculate statistics

In [50]:
# Layers to be used for computing statistics (zonal statistics)
layers_stats = ["POP_PPP","WOCBA_PPP"]
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder'])

The folder '../Results/SEN' has been created


**Get sum for the study area (total population)**

In [51]:
# Initialize a dictionnary for saving total
TOT={}
# Compute sum for the whole study area
for layer in layers_stats:
    TOT["%s"%layer] = float(gscript.parse_command('r.univar', flags='g', map=layer, zones="Study_area")["sum"])
print "Total population of the study area = %s"%TOT["POP_PPP"]
print "Total Women of Child-Bearing Age (WOCBA) of the study area = %s"%TOT["WOCBA_PPP"]

Total population of the study area = 7859233.95042
Total Women of Child-Bearing Age (WOCBA) of the study area = 1956102.20679


**Get sum for each isochrones and compute proportion**

In [52]:
# Compute proportions for each isochrone layer
for isochrone in isochrone_layers:
    for layer in layers_stats:
        # Create temp .csv file
        head, tail = os.path.split(gscript.tempfile())
        table = 'tmp_table_%s'%tail.replace(".","_")
        tmp_csv = os.path.join(head,'%s.csv'%table)
        # Compute sum for each isochrone zone
        gscript.run_command('r.univar', flags='t', overwrite=True, map=layer, 
                            zones=isochrone, output=tmp_csv, separator="comma")
        # Import .csv and join
        gscript.run_command('db.in.ogr', overwrite=True, input=tmp_csv, output=table)
        gscript.run_command('v.db.join', map=isochrone, column='cat', 
                            other_table=table, other_column='zone', subset_columns='sum')
        gscript.run_command('v.db.renamecolumn', map=isochrone, column='sum,%s_SUM'%layer)
        # Add TOTAL count
        gscript.run_command('v.db.addcolumn', map=isochrone, columns='%s_TOT double precision'%layer)
        gscript.run_command('v.db.update', map=isochrone, column='%s_TOT'%layer, value=TOT["%s"%layer])
        # Compute proportion
        gscript.run_command('v.db.addcolumn', map=isochrone, columns='%s_PROP double precision'%layer)
        gscript.run_command('v.db.update', map=isochrone, column='%s_PROP'%layer, 
                            value='({layer}_SUM/{layer}_TOT)*100'.format(layer=layer))
    print "Proportion computed for layer '%s'"%isochrone

Proportion computed for layer 'Isochrones_HCL1L2_WC_WS'
Proportion computed for layer 'Isochrones_HCL2_WC_WS'
Proportion computed for layer 'Isochrones_HCL1L2_NC_WS'
Proportion computed for layer 'Isochrones_HCL2_NC_WS'
Proportion computed for layer 'Isochrones_HCL1L2_WC_DS'
Proportion computed for layer 'Isochrones_HCL2_WC_DS'
Proportion computed for layer 'Isochrones_HCL1L2_NC_DS'
Proportion computed for layer 'Isochrones_HCL2_NC_DS'


In [53]:
## Print processing time
print_processing_time(begintime_processing ,"All processing terminate in ")

'All processing terminate in 2 minutes and 40.6 seconds'

# Exports and cleaning mapset

## Export results

In [54]:
# Output folder isochrones
outputfolder_isochrones = os.path.join(config_parameters['outputfolder'],"Isochrones")
# Check and create folder if needed
check_create_dir(outputfolder_isochrones)

The folder '../Results/SEN/Isochrones' has been created


In [55]:
# Export isochrone layers as GeoPackage and attribute table as .csv
for isochrone in isochrone_layers:
    output_gpkg = os.path.join(outputfolder_isochrones,"%s.gpkg"%isochrone)
    output_csv = os.path.join(config_parameters['outputfolder'],"Prop_by_Isochrones_%s.csv"%isochrone[11:])
    gscript.run_command('v.out.ogr', flags='m', overwrite=True,
                        input=isochrone, output=output_gpkg, format="GPKG")
    gscript.run_command('v.out.ogr', flags='m', overwrite=True,
                        input=isochrone, output=output_csv, format="CSV")

## Remove intermediate layers

In [56]:
# Remove all layers not needed anymore
for rast in TMP_rast:
    gscript.run_command('g.remove', flags='fb', type='raster', name=rast)

In [57]:
# Remove all layers not needed anymore
for vect in TMP_vect:
    gscript.run_command('g.remove', flags='fb', type='vector', name=vect)

In [58]:
# Remove all layers not needed anymore
for vect in isochrone_layers:
    gscript.run_command('g.remove', flags='fb', type='vector', name=vect)

## Delete both mapsets

In [59]:
import shutil

# Delete mapset folder in GRASSDATA
shutil.rmtree(os.path.join("..","GRASSDATA"))